In [233]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import tensorflow as tf
import tensorflow_addons as tfa

In [176]:
data_base = pd.read_excel('./big_merge_V2_meteo_SAT.xlsx').drop('Unnamed: 0', axis=1)

PREPROCESSING _ Code base for models temporal predictions

Ici, features engineering (création de nouvelles features à partir de la liste connues):

In [177]:
# adding aridity index
data_base["AI"] = data_base['PRCP_GROWTH'] / data_base['TAVE_GROWTH']
# adding H/D index
data_base["H_D"] = data_base['HAUTEUR_ARBRE'] / data_base['DBH']

### Choix TARGET et Features :

Target :

In [178]:
TARGET = ['SURF_TER_HA'] #exemple

Features :

In [179]:
# --- PAST --- 
# Attention : on peut, logiquement, inclure la TARGET... (connue dans le passé)
cat_strict = ['PRODREG', 'ESPECE_DOM', 'TYP_RAJ_PPL', 'DEG_FERMETURE', 'STR_PPL', 'RELIEF'] #exemple 'PRODREG', 'ESPECE_DOM', 'TYP_RAJ_PPL', 'DEG_FERMETURE', 'STR_PPL', 'RELIEF'
cat_ord_miss = ['HT_VEG'] #exemple 'TAILLE_PPL', 'MELANGE', 'QUAL_STATION', 'TAUX_COUV_RAJ', 'SURF_TROU_AER', 'HT_VEG'
numerics = ['UNIT_ACCR','H_D','AI','SDI', 'AGE_PPL','ALT', 'TIGES_VIV_H', 'SURF_TER_HA', 'FEUILL_PER', 'CONIF_PER','PERF_CROI', 'NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI'] #exemple

# --- FUTURE KNOWN ---
# Attention : logiquement les features, potentiellement connues dans le futur ci-dessous sont aussi répertoriées ci-dessus dans le passé
add_cat_IFN_stable = [] # technos-substituts : 'DEG_FERMETURE', 'STR_PPL', 'ESPECE_DOM', 'DEG_FERMETURE'
add_cat_ord_IFN_stable = [] # technos-substituts : 'DEG_FERMETURE', 'STR_PPL', 'SURF_TROU_AER' 'SURF_TROU_AER'
add_IFN_numerics_stable = [] # technos-substituts : , 'SDI'
# données potentiellement connues car stables par parcelles :
#['LAT', 'LON', 'ALT', 'PRODREG', 'HT_VEG', 'SLOPE25', 'ASPECT25', 'ORIENTATION', 'PERF_CROI', 'QUAL_STATION', 'UNIT_VEG_FINE',
# 'UNIT_VEG_GROS', 'PROCESS_SILVA', 'PERI_CHENAUX', 'PERI_COULEES','PERI_AVALANCH', 'PERI_CHUTES', 'ETAGE', 'ENDOMMAGEMENT','NB_DEGAT_ARBRE']
# + données extrapolées grâce à des technologies prometteuses (satellites ?)....

add_meteo_known = ['PRCP', 'TAVE_AVG',	'TAVE', 'TAVE_GROWTH', 'PRCP_S_S',	'PRCP_G_S']

add_SAT_known = ['NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI']

In [180]:
features_past = numerics + cat_strict + cat_ord_miss
features_future = add_cat_IFN_stable + add_cat_ord_IFN_stable + add_IFN_numerics_stable + add_meteo_known + add_SAT_known

In [181]:
data_LFI1 = data_base.loc[data_base['LFI']=='LFI1',:].sort_values('PARCELLE')
data_LFI2 = data_base.loc[data_base['LFI']=='LFI2',:].sort_values('PARCELLE')
data_LFI3 = data_base.loc[data_base['LFI']=='LFI3',:].sort_values('PARCELLE')
data_LFI4 = data_base.loc[data_base['LFI']=='LFI4',:].sort_values('PARCELLE')

In [182]:
future_feat_names = []
add_cat_strict_feat_names = []
add_cat_ord_feat_names = []
add_numerics_feat_names = []

for cat in features_future:
    lfi2_list = data_LFI2[cat].to_list()
    lfi3_list = data_LFI3[cat].to_list()
    lfi4_list = data_LFI4[cat].to_list()
    data_LFI1[cat + "_f"] = lfi2_list
    data_LFI2[cat + "_f"] = lfi2_list
    data_LFI3[cat + "_f"] = lfi2_list
    future_feat_names.append(cat + '_f')
    if cat in add_cat_ord_IFN_stable:
        add_cat_ord_feat_names.append(cat + '_f')
    elif cat in add_cat_IFN_stable:
        add_cat_strict_feat_names.append(cat + '_f')
    else:
        add_numerics_feat_names.append(cat + '_f')


In [183]:
data_red = pd.concat([data_LFI1, data_LFI2, data_LFI3], axis=0)[features_past + future_feat_names]
Y = pd.concat([data_LFI2[TARGET], data_LFI3[TARGET], data_LFI4[TARGET]], axis = 0)

In [184]:
data_red.head()

,UNIT_ACCR,H_D,AI,SDI,AGE_PPL,ALT,TIGES_VIV_H,SURF_TER_HA,FEUILL_PER,CONIF_PER,...,TAVE_AVG_f,TAVE_f,TAVE_GROWTH_f,PRCP_S_S_f,PRCP_G_S_f,NDVI_f,EVI_f,NDMI_f,NDWI_f,DSWI_f
0,NaN,0.695652,NaN,571,85.0,715.918970,590.0,27.79,NaN,NaN,...,8.8859,9.026806,13.84776,117.278617,51.835496,0.4992,0.0160,0.2705,-0.4653,0.6865
5,NaN,0.647727,NaN,890,140.0,563.829759,400.0,53.38,NaN,NaN,...,8.9951,19.793490,13.83959,122.675652,55.827235,0.5552,0.0149,0.2017,-0.4864,0.6996
9,NaN,0.666667,NaN,489,80.0,564.885846,320.0,26.70,NaN,NaN,...,9.0637,11.671380,13.39843,128.067334,61.309991,0.5633,0.0153,0.1727,-0.5002,0.6852
14,NaN,NaN,NaN,0,2.0,563.551602,0.0,0.00,NaN,NaN,...,8.8805,10.459870,13.15530,130.759873,63.219754,0.6059,0.0197,0.2269,-0.5588,0.8384
19,NaN,0.676471,NaN,377,130.0,539.769096,150.0,23.32,NaN,NaN,...,8.8384,11.544840,13.16893,131.143585,62.702111,0.6012,0.0184,0.2029,-0.5486,0.7902


In [185]:
Y = Y.to_numpy().ravel()

Traitement des données catégorielles ordonnées en numériques (gestion des "-1" éventuels) :

In [186]:
feats_cat_ord = cat_ord_miss + add_cat_ord_feat_names
for cat in feats_cat_ord:
  data_red[cat] = data_red[cat].apply(lambda v : v if v!=-1 else np.nan)

LISTE DES FEATURES NUMERIQUES ET CATEGORIELLES EN VUE DU PREPROCESSING DE MODELE :

In [187]:
numerics_features = numerics + feats_cat_ord + add_numerics_feat_names
feats_cat_strict = cat_strict + add_cat_strict_feat_names

In [188]:
feats_cat_strict

['PRODREG', 'ESPECE_DOM', 'TYP_RAJ_PPL', 'DEG_FERMETURE', 'STR_PPL', 'RELIEF']

In [189]:
numerics_transforms = Pipeline(
    [('imputer',KNNImputer()),
    ('encoder',StandardScaler())
])
categorials_transforms = Pipeline([
    ('imputer',KNNImputer()),
    ('encoder',OneHotEncoder(drop="first"))
])

preprocessor = ColumnTransformer(
    [("num", numerics_transforms, numerics_features),
     ("cat", categorials_transforms, feats_cat_strict)])

In [19]:
#data_red['ORIENTATION'] = data_red['ORIENTATION'].map({'N':0,'NE':1,'E':2,'SE':3,'S':4,'SO':5,'O':6,'NO':7})
#data_red['ORIENTATION_f'] = data_red['ORIENTATION_f'].map({'N':0,'NE':1,'E':2,'SE':3,'S':4,'SO':5,'O':6,'NO':7})


In [190]:
X_train, X_test, y_train, y_test = train_test_split(data_red, Y, test_size=0.2, random_state=2)

In [191]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [192]:
np.shape(X_train)

(5767, 67)

In [193]:
np.shape(X_test)

(1442, 67)

In [194]:
model =  Ridge(max_iter=10000)

In [195]:
params = {
    'alpha':[110, 120, 130]
}

In [196]:
grid = GridSearchCV(model, param_grid=params, scoring='r2', verbose=2)

In [197]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ..........................................alpha=110; total time=   0.0s
[CV] END ..........................................alpha=110; total time=   0.0s
[CV] END ..........................................alpha=110; total time=   0.0s
[CV] END ..........................................alpha=110; total time=   0.0s
[CV] END ..........................................alpha=110; total time=   0.0s
[CV] END ..........................................alpha=120; total time=   0.0s
[CV] END ..........................................alpha=120; total time=   0.0s
[CV] END ..........................................alpha=120; total time=   0.0s
[CV] END ..........................................alpha=120; total time=   0.0s
[CV] END ..........................................alpha=120; total time=   0.0s
[CV] END ..........................................alpha=130; total time=   0.0s
[CV] END ........................................

GridSearchCV(estimator=Ridge(max_iter=10000),
             param_grid={'alpha': [110, 120, 130]}, scoring='r2', verbose=2)

In [198]:
grid.best_estimator_

Ridge(alpha=110, max_iter=10000)

In [199]:
train_scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5)
test_scores = cross_val_score(grid.best_estimator_, X_test, y_test, cv=5)
print(f'Train score mean : {np.mean(train_scores)}')
print(f'Train score std : {np.std(train_scores)}')
print(f'Test score mean : {np.mean(test_scores)}')
print(f'Test score std : {np.std(test_scores)}')

Train score mean : 0.6567868522982752
Train score std : 0.038255169075672564
Test score mean : 0.6849304953755128
Test score std : 0.037340866306863646


In [200]:
list_features_in = []
for feat in numerics_features:
  list_features_in.append(feat)
for cat in feats_cat_strict:
  nb_lab = len(data_red[cat].unique())-1
  for i in range(nb_lab):
    list_features_in.append(f'{cat}_{i}')

In [201]:
list_features_in

['UNIT_ACCR',
 'H_D',
 'AI',
 'SDI',
 'AGE_PPL',
 'ALT',
 'TIGES_VIV_H',
 'SURF_TER_HA',
 'FEUILL_PER',
 'CONIF_PER',
 'PERF_CROI',
 'NDVI',
 'EVI',
 'NDMI',
 'NDWI',
 'DSWI',
 'HT_VEG',
 'PRCP_f',
 'TAVE_AVG_f',
 'TAVE_f',
 'TAVE_GROWTH_f',
 'PRCP_S_S_f',
 'PRCP_G_S_f',
 'NDVI_f',
 'EVI_f',
 'NDMI_f',
 'NDWI_f',
 'DSWI_f',
 'PRODREG_0',
 'PRODREG_1',
 'PRODREG_2',
 'PRODREG_3',
 'ESPECE_DOM_0',
 'ESPECE_DOM_1',
 'ESPECE_DOM_2',
 'ESPECE_DOM_3',
 'ESPECE_DOM_4',
 'ESPECE_DOM_5',
 'ESPECE_DOM_6',
 'ESPECE_DOM_7',
 'ESPECE_DOM_8',
 'ESPECE_DOM_9',
 'ESPECE_DOM_10',
 'ESPECE_DOM_11',
 'TYP_RAJ_PPL_0',
 'TYP_RAJ_PPL_1',
 'TYP_RAJ_PPL_2',
 'DEG_FERMETURE_0',
 'DEG_FERMETURE_1',
 'DEG_FERMETURE_2',
 'DEG_FERMETURE_3',
 'DEG_FERMETURE_4',
 'DEG_FERMETURE_5',
 'DEG_FERMETURE_6',
 'DEG_FERMETURE_7',
 'DEG_FERMETURE_8',
 'DEG_FERMETURE_9',
 'DEG_FERMETURE_10',
 'STR_PPL_0',
 'STR_PPL_1',
 'STR_PPL_2',
 'STR_PPL_3',
 'RELIEF_0',
 'RELIEF_1',
 'RELIEF_2',
 'RELIEF_3',
 'RELIEF_4']

In [202]:
df_coef = pd.DataFrame(grid.best_estimator_.coef_, columns=['Coeff'], index= list_features_in)

In [203]:
df_coef

,Coeff
UNIT_ACCR,0.194342
H_D,-0.624101
AI,-0.537192
SDI,4.508372
AGE_PPL,0.130372
...,...
RELIEF_0,0.755609
RELIEF_1,-0.591321
RELIEF_2,-0.536332
RELIEF_3,0.105369


In [204]:
fig = px.bar(df_coef['Coeff'], title=f"Features importance for target : {TARGET} with Lasso Linear Regression")
fig.show()

-----------------

In [207]:
train_batch = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train)).batch(batch_size=32)
test_batch = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size=32)

In [209]:
np.shape(X_train)

(5767, 67)

In [208]:
for x, y in train_batch.take(1):
    print(y)

tf.Tensor(
[41.23  7.5  20.81  4.28 23.29 10.37 40.57 56.68 44.88 76.25 33.68  8.28
 18.58 41.9  32.65  4.54 44.91 34.88 16.05 35.9  16.83 46.99 50.94 15.12
 53.22 66.58 45.69  0.   12.15 29.14 30.73 20.79], shape=(32,), dtype=float64)


In [210]:
model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(np.shape(X_train)[1],)),
        tf.keras.layers.Dense(32,"relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16,"relu"),
        tf.keras.layers.Dense(8,"relu"),
        tf.keras.layers.Dense(1)
    ])

In [212]:
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=tfa.metrics.RSquare())

In [213]:
model.fit(train_batch, epochs=20, validation_data=test_batch)

Epoch 1/20
181/181 [==============================] - 4s 12ms/step - loss: 950.1282 - r_square: -1.3445 - val_loss: 185.3095 - val_r_square: 0.4907
Epoch 2/20
181/181 [==============================] - 2s 11ms/step - loss: 194.2164 - r_square: 0.5208 - val_loss: 133.3752 - val_r_square: 0.6334
Epoch 3/20
181/181 [==============================] - 2s 11ms/step - loss: 173.8271 - r_square: 0.5711 - val_loss: 126.7428 - val_r_square: 0.6517
Epoch 4/20
181/181 [==============================] - 2s 10ms/step - loss: 168.4070 - r_square: 0.5845 - val_loss: 122.0445 - val_r_square: 0.6646
Epoch 5/20
181/181 [==============================] - 2s 10ms/step - loss: 159.4179 - r_square: 0.6066 - val_loss: 119.8295 - val_r_square: 0.6707
Epoch 6/20
181/181 [==============================] - 2s 9ms/step - loss: 160.4852 - r_square: 0.6040 - val_loss: 119.8649 - val_r_square: 0.6706
Epoch 7/20
181/181 [==============================] - 2s 10ms/step - loss: 156.0993 - r_square: 0.6148 - val_loss: 117

In [226]:
coeff_mean = []
for i in range(67):
    coeff_mean.append(np.mean(model.layers[0].trainable_variables[0][i]))


In [229]:
df_coef = pd.DataFrame(coeff_mean, columns=['Coeff'], index= list_features_in)

In [230]:
fig = px.bar(df_coef['Coeff'], title=f"Features importance for target : {TARGET} with Lasso Linear Regression")
fig.show()

In [235]:
y_dum = np.mean(Y) * np.ones(7209)

In [238]:
y_dum

array([33.350086, 33.350086, 33.350086, ..., 33.350086, 33.350086,
       33.350086])

In [234]:
len(Y)

7209

In [239]:
r2_score(Y, y_dum)

0.0